In [6]:
import numpy as np

In [115]:
nan = np.nan

In [116]:
#we define the transition probability for each action at each state.

T = np.array([
    [[0.7, 0.3, 0], [1, 0,0], [0.8, 0.2, 0]],
    [[0, 1, 0], [nan, nan, nan], [0, 0, 1]],
    [[nan, nan, nan], [0.8, 0.1, 0.1], [nan, nan, nan]],
])

In [117]:
#we define the reward while performing an each action at each state.

R = np.array([
    [[10, 0, 0], [0, 0, 0], [0, 0, 0]],
    [[0, 0, 0], [nan, nan, nan], [0, 0, -50]],
    [[nan, nan, nan], [40, 0, 0], [nan, nan, nan]],
])

In [118]:
#possible action that we can perform at each state 
possible_actions = [[0,1,2],[0,2], [1]]
#disount factor
lr = 0.95
#number of iteration
n_iterate = 1000

In [132]:
#function to define V-value of each state.
'''We start by initialize the value funtion at each state with impossible'''
def V_values(lr, n_iterate, T, R, possible_actions):
    '''
    Inputs: lr dicount factor
            n_iterate number of iteration
            T table of transition probavility
            R table of reward
            possible_actions different actions that can be take at each state.
    Return V vector of Q values
    '''
    V = np.full(3, -np.inf)
    for state, actions in enumerate(possible_actions):
        V[state] = 0.0
    for i in range(n_iterate):
        V_prev = V.copy()
        for s in range(3):
            V[s] = np.max([np.sum([
                T[s,a,sp]*(R[s,a,sp] + lr*(V_prev[sp])) for sp in range(3)
            ]) for a in possible_actions[s]
                          ])
    return V

In [133]:
# np.max([np.sum([] for sp in range(3))] for a in possible_actions[s])

In [134]:
V_values(lr,n_iterate, T, R, possible_actions)

array([21.89925005,  1.17982024, 53.87349498])

In [135]:
def Q_values(lr, n_iterate, T, R, possible_actions):
    '''
    Inputs: lr dicount factor
            n_iterate number of iteration
            T table of transition probavility
            R table of reward
            possible_actions different actions that can be take at each state.
    Return Q matrix of Q values
    '''
    Q = np.full((3,3), -np.inf)
    for state, actions in enumerate(possible_actions):
        Q[state, actions] = 0.0
    for i in range(n_iterate):
        Q_prev = Q.copy()
        for s in range(3):
            for a in possible_actions[s]:
                Q[s,a] = np.sum([
                    T[s,a,sp]*(R[s,a,sp] + lr*np.max(Q_prev[sp])) for sp in range(3)
                ])
    return Q

**By runing the Q-values iterative algorithm, we find the following result:**

In [131]:
Q_values(lr,n_iterate, T, R, possible_actions)

array([[21.89925005, 20.80428755, 16.86759588],
       [ 1.12082922,        -inf,  1.17982024],
       [       -inf, 53.87349498,        -inf]])

**This algorithm is verry necessary in reinforcement learning because it gives the value at each state while perfoming a specific action. Since the goal here is to find the optimal policy that can maximize the cumulative rewards, by taking the argmax() function ones will best actions to take.**

In [47]:
best_act = np.argmax(Q, axis = 1)
print('The best action to take at each state is {}.'.format(best_act))

The best action to take at each state is [0 2 1]


**Temporal difference learning algorithm**
RL problem with discret actions can be modeled using MDP. in TD, the agent has partial information about the MDP. In generale, we assume that the agent initially know the states and actions to take.

In [1]:
lr_rate0= 0.05
lr_rate_decay=0.1
gamma= 0.95
n_iterates = 10000000

def V_value_TD(lr_rate0, lr_rate_decay, gamma, T, R, n_iterates):
    V = np.full(3, -np.inf)
    for state, actions in enumerate(possible_actions):
        V[state] = 0.0
    s = 0
    for iterate in range(n_iterates):
        V_prev = V.copy()
        lr_rate = lr_rate0 / (1 + iterate*lr_rate_decay)
        a = np.random.choice(possible_actions[s])
        sp = np.random.choice(range(3), p = T[s,a])
        r = R[s,a,sp]
        V[s] = (1 - lr_rate)*V_prev[s] + lr_rate*(r + gamma*V_prev[sp])
        s = sp
    return V
        
        

In [63]:
V_value_TD(lr_rate0, lr_rate_decay, gamma, T, R, n_iterates)

array([ -0.14950054, -20.81673127,  19.51785965])

In [64]:
def Q_value_TD(lr_rate0, lr_rate_decay, gamma, T, R, n_iterates):
    Q = np.full((3,3), -np.inf)
    s = 0
    for state, actions in enumerate(possible_actions):
        Q[state, actions] = 0
    for iterate in range(n_iterates):
        Q_prev = Q.copy()
        lr_rate = lr_rate0 / (1 + iterate*lr_rate_decay)
        a = np.random.choice(possible_actions[s])
        sp = np.random.choice(range(3), p = T[s,a])
        r = R[s,a,sp]
        Q[s,a] = (1 - lr_rate)*Q_prev[s,a] + lr_rate*(r + gamma*np.max(Q_prev[sp]))
        s = sp
    return Q

In [65]:
Q_value = Q_value_TD(lr_rate0, lr_rate_decay, gamma, T, R, n_iterates)
Q_value

array([[  7.96310973,   3.73589636,   2.99749087],
       [  0.        ,         -inf, -21.04476803],
       [        -inf,  22.15838247,         -inf]])

In [66]:
#Optimal policy
np.argmax(Q_value, axis = 1)

array([0, 0, 1])

In [5]:
#!pip3 install gym


**Implementation of Q-learning using gym enveronment.**

In [110]:
import gym
import torch
import time
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from gym.envs.registration import register
from IPython.display import clear_output
try:
    register(
        id='FrozenLakeNolip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

env_names = 'FrozenLakeNolip-v0'
env = gym.make(env_names)
print('Observation spaces:', env.observation_space)
print('Actions spaces:', env.action_space)
type(env.action_space)


Observation spaces: Discrete(16)
Actions spaces: Discrete(4)


gym.spaces.discrete.Discrete

In [96]:
class Qagent():
    def __init__(self, env):
        self.action_size = env.action_space.n

    def get_action(self, state):
        action = np.random.choice(range(self.action_size))
        return action

In [113]:
class Agent(Qagent):
    def __init__(self, env, discount_rate= 0.95, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.eps = 1.0
        self.model()
        
    def model(self):
        self.q_value = 1e-5 * np.random.random((self.state_size, self.action_size))
        
    def get_action(self, state):
        #we first get the row containing the q_value for the given state.
        q_state = self.q_value[state]
        #exploitation
        action_greedy = np.argmax(q_state)
        #exploiration 
        action_random = super().get_action(state)
        
        return action_random if np.random.random() < self.eps else action_greedy
    
    
    def train(self, lis):
        state, action, next_state, reward, done= lis
        q_next =    self.q_value[next_state]
        q_next =    np.zeros(self.action_size) if done else q_next
        q_target =  reward + self.discount_rate * np.max(q_next)
        q_update =  q_target - self.q_value[state,action]
        self.q_value[state,action]  +=  self.learning_rate * q_update
        
        
        if done:
            self.eps = self.eps * 0.99
            
agent = Agent(env)       

In [114]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state, action, next_state, reward, done))
        state = next_state
        total_reward += reward
        print('s:', state, 'a:',action, 'reward', reward)
        print('Episode {}, Total rewards {}, eps {}'.format(ep,total_reward, agent.eps))
        env.render()
        print(agent.q_value)
        time.sleep(0.02)
        clear_output(wait = True)

s: 15 a: 2 reward 1.0
Episode 99, Total rewards 13.0, eps 0.36603234127322926
  (Right)
SFFF
FHFH
FFFH
HFFG
[[5.97170587e-06 7.20643562e-06 5.20966258e-06 2.85651206e-06]
 [4.25270648e-06 5.93326848e-06 6.35704204e-06 4.93587023e-06]
 [1.78204483e-06 4.96594090e-06 3.86067345e-06 5.47007061e-06]
 [1.79761676e-06 6.11245666e-06 4.99850984e-06 3.66892095e-06]
 [2.65504517e-06 7.06235342e-06 2.61266295e-06 7.54317372e-06]
 [9.79798540e-06 3.40974522e-07 6.76267624e-06 6.96989198e-06]
 [5.94996322e-06 6.40421567e-05 7.67579135e-06 8.02206002e-06]
 [1.95227846e-06 7.42049414e-06 6.64414663e-06 9.53822404e-06]
 [5.97750779e-06 6.85830100e-06 1.28469866e-05 3.97660882e-06]
 [3.57804951e-06 6.00256655e-06 2.24217352e-04 9.39374950e-07]
 [5.36911201e-06 6.37624983e-03 1.03903169e-06 2.00314731e-06]
 [6.21341162e-06 3.23348402e-06 2.76927891e-06 3.70475141e-06]
 [7.99076382e-06 1.87016958e-06 4.99258549e-06 5.43836616e-07]
 [8.21809378e-06 1.52950597e-05 2.36137633e-03 2.54532904e-07]
 [2.481406